### 本例程开始搭建深度神经网络之一的卷积神经网络.
### 卷积神经网络由多个卷积层构成. 每个卷积层进行如下操作:
(1)图像通过多个卷积核进行卷积操作,提取出局部特征,每个卷积核会映射出一个新的2D图像.并对输出结果进行Relu等激活函数处理.
(2)池化,减少参数.最常采用最大池化以保留最显著的特征.


In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_DATA",one_hot = True)

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [7]:
# 创建会话.后续过程都在该会话中进行
sess = tf.InteractiveSession()

# 深度神经网络权值较多,首先定义一个初始化函数.为克服梯度为零和死亡节点的问题,将初始权重设为正态分布,偏置设为0.1
def weight_variable (shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable (shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

# 同样,卷积和池化操作也是多次用到,也定义一下
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding = 'SAME')
# 数字代表卷积模板移动的步长.padding为边界处理方式,'same'代表输入和输出保持同样尺寸
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
# 池化并缩小尺寸,横竖步长都为2

# 充分利用图像结构信息,将输入尺寸变为28*28*1,1代表通道数量
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])
x_reshape = tf.reshape(x,[-1,28,28,1])

# 第一个卷积层
w_conv1 = weight_variable([5,5,1,32])  #卷积核尺寸为5*5,1个颜色通道,32个卷积核即32个特征
b_conv1 = bias_variable([32])
y_conv1 = tf.nn.relu(conv2d(x_reshape,w_conv1) + b_conv1) #卷积和relu操作
y_pool1 = max_pool_2x2(y_conv1)   #池化操作


# 第二个卷积层
w_conv2 = weight_variable([5,5,32,64])  #卷积核尺寸为5*5,64个卷积核即64个特征
b_conv2 = bias_variable([64])
y_conv2 = tf.nn.relu(conv2d(y_pool1,w_conv2) + b_conv2) #卷积和relu操作
y_pool2 = max_pool_2x2(y_conv2)   #池化操作

# 全连接层
# 经过两次2*2的池化操作,特征数量为64,则输出尺寸为7*7*64.卷积核定为1024个
# 在全连接层前需要将数据转化为1维的.
w_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
y_pool2 = tf.reshape(y_pool2,[-1,7*7*64])
y_fc1 = tf.nn.relu(tf.matmul(y_pool2,w_fc1) + b_fc1)

# 添加dropout.一般来说,dropout常加在全连接层
drop_out = tf.placeholder(tf.float32)
y_fc1_drop = tf.nn.dropout(y_fc1,drop_out)

# 输出层.仍使用softmax函数
w_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y = tf.nn.softmax(tf.matmul(y_fc1_drop,w_fc2) + b_fc2)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

# 将SGD寻优算法改为Adam.learning_rate要通过不断调试才能找到最合适的值.
learning_rate = 0.0001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# 全局参数初始化
tf.global_variables_initializer().run()

# 设置准确率函数.当预测值与实际值相同时,即为分类准确.然后求整体的准确率
correct_pred = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

# 喂参数,开始训练.每次随机抽取100条样本进行训练,共迭代1000次.注意:输入的y值是真实标签,应当是y_! y是输出值
for i in range(20000):
    batch_x, batch_y = mnist.train.next_batch(50)
    optimizer.run({x:batch_x,y_:batch_y,drop_out:0.5})   #训练时drop_out设置为0.5

    if i%500 == 0:
        print "迭代"+ str(i) + "次,训练准确率为:" + str(acc.eval({x:batch_x,y_:batch_y,drop_out:1.0}))
        
#     如上的run()操作也可写为:sess.run(optimizer,feed_dict={x:batch_x,y_:batch_y})
#                        与:sess.run(acc,feed_dict={x:batch_x,y_:batch_y})
#     这样看起来更清晰.在执行算法定义的代码时,计算并没有发生.只有调用run方法并feed数据时才真正执行.

# 由于内存限制,只选取了测试集中的2000张进行测试
batch_xt, batch_yt = mnist.test.next_batch(2000)
print "整体测试准确率为:" + str(acc.eval(feed_dict = {x:batch_xt,y_:batch_yt,drop_out:1.0}))   #测试时drop_out设置为1.0

迭代0次,训练准确率为:0.12
迭代500次,训练准确率为:0.86
迭代1000次,训练准确率为:0.94
迭代1500次,训练准确率为:0.96
迭代2000次,训练准确率为:0.98
迭代2500次,训练准确率为:0.96
迭代3000次,训练准确率为:0.98
迭代3500次,训练准确率为:0.96
迭代4000次,训练准确率为:1.0
迭代4500次,训练准确率为:1.0
迭代5000次,训练准确率为:1.0
迭代5500次,训练准确率为:0.98
迭代6000次,训练准确率为:1.0
迭代6500次,训练准确率为:0.98
迭代7000次,训练准确率为:1.0
迭代7500次,训练准确率为:0.98
迭代8000次,训练准确率为:1.0
迭代8500次,训练准确率为:0.96
迭代9000次,训练准确率为:1.0
迭代9500次,训练准确率为:1.0
迭代10000次,训练准确率为:0.98
迭代10500次,训练准确率为:0.98
迭代11000次,训练准确率为:1.0
迭代11500次,训练准确率为:1.0
迭代12000次,训练准确率为:1.0
迭代12500次,训练准确率为:1.0
迭代13000次,训练准确率为:1.0
迭代13500次,训练准确率为:0.96
迭代14000次,训练准确率为:1.0
迭代14500次,训练准确率为:1.0
迭代15000次,训练准确率为:1.0
迭代15500次,训练准确率为:1.0
迭代16000次,训练准确率为:0.98
迭代16500次,训练准确率为:0.98
迭代17000次,训练准确率为:1.0
迭代17500次,训练准确率为:1.0
迭代18000次,训练准确率为:1.0
迭代18500次,训练准确率为:1.0
迭代19000次,训练准确率为:1.0
迭代19500次,训练准确率为:1.0
整体测试准确率为:0.986
